In [2]:
import model
import tensorflow as tf
import os
import numpy as np
import data_train_test
import matplotlib.pyplot as plt


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:

siamese = model.make_siamese()
binary_cross_entropy = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(0.001)

#creating checkpoints
filepath = "\ml_projects\smart_attendance\checkpoints"
if(not os.path.exists(filepath)):
    os.makedirs(filepath)
checkpoint_prefix = os.path.join(filepath,"ckpt")
check_point  = tf.train.Checkpoint(opt = opt , siamese = siamese)
tf.executing_eagerly()

@tf.function
def train_step(batch):

    with tf.GradientTape() as g:

        X = batch[:2]
        y = batch[2]

        y_hat = siamese(X,training= True)
        loss = binary_cross_entropy(y,y_hat)
    
    gradient = g.gradient(loss,siamese.trainable_variables)
    opt.apply_gradients(zip(gradient,siamese.trainable_variables))

    return loss

def train_model(data,epochs):

    for epoch in range(1,epochs+1):
        print("/n Epoch {}/{}".format(epoch,epochs))
        progbar = tf.keras.utils.Progbar(len(data))
        for index,batch in enumerate(data):
            train_step(batch)
            progbar.update(index+1)
        if epoch % 5 == 0:
            check_point.save(file_prefix=checkpoint_prefix)


data_train,data_test = data_train_test.getting_data()
train_model(data_train,50)
siamese.save("siamese1.h5")